In [0]:
!pip install mpld3

     |████████████████████████████████| 798kB 2.9MB/s 
  Created wheel for mpld3: filename=mpld3-0.3-cp36-none-any.whl size=116679 sha256=3baa1965ccf575640457245660f075a72787f10333cfc12a4060ca1334857c8a
  Stored in directory: /root/.cache/pip/wheels/c0/47/fb/8a64f89aecfe0059830479308ad42d62e898a3e3cefdf6ba28
Successfully built mpld3


In [0]:
import numpy as np
import pandas as pd
import nltk
import re
import os
import codecs
from sklearn import feature_extraction
import mpld3
from bs4 import BeautifulSoup
import urllib3
from nltk.stem.snowball import SnowballStemmer
from sklearn.decomposition import PCA
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.cluster import KMeans, AffinityPropagation
import plotly as ptl
import plotly.express as px

In [0]:
urls_data = [
    ("titles_urls", "https://github.com/brandomr/document_cluster/raw/master/title_list.txt", "\n"),
    ("link_list_imdb_url", "https://github.com/brandomr/document_cluster/raw/master/link_list_imdb.txt", "\n"),
    ("genrne_list_url", "https://github.com/brandomr/document_cluster/raw/master/genres_list.txt", "\n"),
    ("synopses_wiki_url", "https://github.com/brandomr/document_cluster/raw/master/synopses_list_wiki.txt", "\n BREAKS HERE"),
    ("synopses_imdb_url", "https://github.com/brandomr/document_cluster/raw/master/synopses_list_imdb.txt", "\n BREAKS HERE")
]

nltk.download('punkt')
stemmer = SnowballStemmer("english")

urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)

pool = urllib3.PoolManager()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
dataframe = pd.DataFrame()

for title, url, splitter in urls_data:
    dataframe[title] = pool.request('GET', url).data.decode('utf-8').split(splitter)

dataframe.dropna(inplace=True)
dataframe = dataframe.assign(
    synopses_wiki_url=lambda row: row.synopses_wiki_url.map(lambda text: BeautifulSoup(text, "html.parser").text),
    synopses_imdb_url=lambda row: row.synopses_imdb_url.map(lambda text: BeautifulSoup(text, "html.parser").text),
    synopses=lambda row: row.synopses_wiki_url + row.synopses_imdb_url
)

In [0]:
dataframe

,titles_urls,link_list_imdb_url,genrne_list_url,synopses_wiki_url,synopses_imdb_url,synopses
0,The Godfather,http://www.imdb.com/title/tt0068646/,"[u' Crime', u' Drama']",Plot [edit] [ [ edit edit ] ] \n On ...,"\n\nIn late summer 1945, guests are gathered f...",Plot [edit] [ [ edit edit ] ] \n On ...
1,The Shawshank Redemption,http://www.imdb.com/title/tt0111161/,"[u' Crime', u' Drama']",Plot [edit] [ [ edit edit ] ] \n In ...,"\n\nIn 1947, Andy Dufresne (Tim Robbins), a ba...",Plot [edit] [ [ edit edit ] ] \n In ...
2,Schindler's List,http://www.imdb.com/title/tt0108052/,"[u' Biography', u' Drama', u' History']",Plot [edit] [ [ edit edit ] ] \n In ...,\n\nThe relocation of Polish Jews from surroun...,Plot [edit] [ [ edit edit ] ] \n In ...
3,Raging Bull,http://www.imdb.com/title/tt0081398/,"[u' Biography', u' Drama', u' Sport']",Plot [edit] [ [ edit edit ] ] \n In ...,"\n\nThe film opens in 1964, where an older and...",Plot [edit] [ [ edit edit ] ] \n In ...
4,Casablanca,http://www.imdb.com/title/tt0034583/,"[u' Drama', u' Romance', u' War']",Plot [edit] [ [ edit edit ] ] \n It ...,"\n\nIn the early years of World War II, Decemb...",Plot [edit] [ [ edit edit ] ] \n It ...
...,...,...,...,...,...,...
96,Rear Window,http://www.imdb.com/title/tt0047396/,"[u' Mystery', u' Thriller']",Plot [edit] [ [ edit edit ] ] \n \n\...,"\n\nL.B. ""Jeff"" Jeffries (James Stewart) recup...",Plot [edit] [ [ edit edit ] ] \n \n\...
97,The Third Man,http://www.imdb.com/title/tt0041959/,"[u' Film-Noir', u' Mystery', u' Thriller']",Plot [edit] [ [ edit edit ] ] \n \n\...,"\n\nSights of Vienna, Austria, flash across th...",Plot [edit] [ [ edit edit ] ] \n \n\...
98,North by Northwest,http://www.imdb.com/title/tt0053125/,"[u' Mystery', u' Thriller']",Plot [edit] [ [ edit edit ] ] \n Adv...,"\n\nAt the end of an ordinary work day, advert...",Plot [edit] [ [ edit edit ] ] \n Adv...
99,Yankee Doodle Dandy,http://www.imdb.com/title/tt0035575/,"[u' Biography', u' Drama', u' Musical']",Synopsis [edit] [ [ edit edit ] ] \n ...,\n,Synopsis [edit] [ [ edit edit ] ] \n ...


In [0]:
stemmer = SnowballStemmer("english")

def tokenize_and_stem(text):
    return [stemmer.stem(t) for t in tokenize_only(text)]

def tokenize_only(text):
    tokens = [word.lower() for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent)]
    return filter(lambda t: re.search('[a-zA-Z]', t), tokens)

In [0]:
totalvocab_stemmed = [word for elem in dataframe["synopses"] for word in tokenize_and_stem(elem)]
totalvocab_tokenized = [word for elem in dataframe["synopses"] for word in tokenize_only(elem)]

In [0]:
vocab_frame = pd.DataFrame({'words': totalvocab_tokenized}, index = totalvocab_stemmed)

In [0]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vectorizer = TfidfVectorizer(
    max_df=0.8, 
    min_df=0.2, 
    tokenizer=tokenize_and_stem, 
    ngram_range=(1,3)
)

In [0]:
tfidf_matrix = tfidf_vectorizer.fit_transform(dataframe["synopses"])
terms = tfidf_vectorizer.get_feature_names()

In [0]:
tfidf_dataframe = pd.DataFrame(tfidf_matrix.todense())

In [0]:
n_components = 100

pca = PCA(n_components)

pca_results = pd.DataFrame(pca.fit_transform(tfidf_dataframe))

for index, columns_name in enumerate(pca_results):
   tfidf_dataframe[f"PC{index+1}"] = pca_results[columns_name] 

In [0]:
num_clusters = 5
final_dataframe = tfidf_dataframe.assign(
    km_cluster_pca=KMeans(num_clusters).fit_predict(tfidf_dataframe[[f"PC{index+1}" for index in range(n_components)]]),
    affinity_cluster_pca=AffinityPropagation().fit_predict(tfidf_dataframe[[f"PC{index+1}" for index in range(n_components)]]),
    titles = dataframe['titles_urls']
)

In [0]:
px.scatter_3d(
    final_dataframe,
    x='PC1', 
    y='PC2',
    z='PC3',
    hover_name='titles',
    symbol='km_cluster_pca',
    color='km_cluster_pca'
)

In [0]:
px.scatter_3d(
    final_dataframe,
    x='PC1', 
    y='PC2',
    z='PC3',
    hover_name='titles',
    symbol='affinity_cluster_pca',
    color='affinity_cluster_pca'
)